# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# check the current working directory
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)    
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating the full data CSV
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data modeling with Apache Cassandra

## Dataset

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appear like in the `event_datafile_new.csv` after the code above is run:

<img src="images/image_event_datafile_new.jpg"></img>

### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_session
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_session')
except Exception as e:
    print(e)

## Create queries to deal with the following three questions:

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Query 1

In [11]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4

# Partition key: (session_id, item_in_session)
try:
    session.execute("""
    CREATE TABLE session_item 
    (
        session_id INT,
        item_in_session INT,
        artist VARCHAR,
        song_title VARCHAR,
        length FLOAT,
        PRIMARY KEY(session_id, item_in_session)
    );"""
)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [17]:
try:
    rows = session.execute("SELECT * FROM session_item WHERE session_id=338 AND item_in_session=4")
except Exception as e:
    print(e)
    
query1_ans = pd.DataFrame([row.session_id, row.item_in_session, row.artist, row.song_title, row.length] for row in rows)
query1_ans.columns=['sessionId','itemInSession', 'artist', 'song', 'length']
query1_ans

,sessionId,itemInSession,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2

In [16]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Partition key: (user_id, session_id)
# Clustering column: item_in_session
try:
    session.execute("""
    CREATE TABLE user_session
    (
        user_id INT,
        session_id INT,
        item_in_session INT,
        song VARCHAR,
        first_name VARCHAR,
        last_name VARCHAR,
        PRIMARY KEY((user_id, session_id), item_in_session)
    );"""
)
except Exception as e:
    print(e)

# INSERT
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (user_id, session_id, item_in_session, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[1], line[4]))
        
# SELECT
try:
    rows = session.execute("SELECT * FROM user_session WHERE user_id = 10 AND session_id = 182")
except Exception as e:
    print(e)
      
query2_ans = pd.DataFrame([row.user_id, row.session_id, row.item_in_session, row.song, row.first_name, row.last_name] for row in rows)
query2_ans.columns=['userId','sessionId','itemInSession', 'song', 'firstName', 'lastName']
query2_ans

,userId,sessionId,itemInSession,song,firstName,lastName
0,10,182,0,Keep On Keepin' On,Sylvie,Cruz
1,10,182,1,Greece 2000,Sylvie,Cruz
2,10,182,2,Kilometer,Sylvie,Cruz
3,10,182,3,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3

In [19]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Partition key: (song_title, user_id)
try:
    session.execute("""
    CREATE TABLE user_song
    (
        first_name VARCHAR,
        last_name VARCHAR,
        song VARCHAR,
        user_id INT,
        PRIMARY KEY(song, user_id)
    );"""
)
except Exception as e:
    print(e)

# INSERT
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (user_id, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))
        
# SELECT
try:
    rows = session.execute("SELECT * FROM user_song WHERE song = 'All Hands Against His Own'")
except Exception as e:
    print(e)
    

query3_ans = pd.DataFrame([row.song, row.user_id, row.first_name, row.last_name] for row in rows)
query3_ans.columns=['song', 'userId', 'firstName', 'lastName']
query3_ans

,song,userId,firstName,lastName
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson


### Drop the tables before closing out the sessions

In [10]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS session_item")
    session.execute("DROP TABLE IF EXISTS user_session")
    session.execute("DROP TABLE IF EXISTS user_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()